<a href="https://colab.research.google.com/github/kyletran77/Coursera_Capstone/blob/master/Week_3.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1 . Import the libraries to scrape the website**

In [0]:
import pandas as pd   #import the libraries we need
from bs4 import BeautifulSoup  #lib for table html
import requests

**2. Use the wiki url and parse data into BeautifulSoup**

In [0]:
url  = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html5lib') #get wiki

In [0]:
postallist = [] #list for each cell
boroughlist = []
neighbourhood = []

**3.  Locate the rows from table and update each row into the lists.**

In [0]:
soup.find('table').find_all('tr')#table
soup.find('table').find_all('td')#rows
for row in soup.find('table').find_all('tr'):
  cells = row.find_all('td')#find all rows in table
  if(len(cells)>0):
    postallist.append(cells[0].text)
    boroughlist.append(cells[1].text)
    neighbourhood.append(cells[2].text.rstrip('\n'))# strip the ending to prevent newlines

**4. Create a dataframe**

In [6]:
tor_df = pd.DataFrame({'PostalCode': postallist,
                      'Borough':boroughlist,
                      'Neighborhood':neighbourhood}) #create the dataframe 
tor_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [7]:
tor_df1 = tor_df[tor_df['Borough'] != 'Not assigned'].reset_index(drop=True)
tor_df1.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [8]:
tor_group=tor_df1.groupby(['PostalCode', 'Borough'], as_index = False).agg(lambda x: ','.join(x))
tor_group.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


**5. Organize table to match question’s values**

In [9]:
columns_ = ['PostalCode','Borough','Neighborhood']
test_d = pd.DataFrame(columns= columns_)#create test dataframe

test_list = ['M5G','M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R','M9V','M9L','M5V','M1B', 'M5A']

for code in test_list:#update test dataframe where it takes each value in the test list to get the value
  test_d = test_d.append(tor_group[tor_group['PostalCode']==code], ignore_index = True)
test_d



,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens,Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale,Wexford"
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."


In [10]:
tor_group.shape

(103, 3)

In [25]:
!wget -O Geospatial_Coordinates.csv 'https://cocl.us/Geospatial_data'

--2019-08-06 00:02:58--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-08-06 00:02:59--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-08-06 00:03:00--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

In [27]:
location = pd.read_csv("Geospatial_Coordinates.csv")
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
location.rename(columns={'Postal Code':'PostalCode'}, inplace = True)
location.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
tor_location = tor_group.merge(location, on = 'PostalCode', how = 'left')
tor_location.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [40]:
columnname = ['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude']
test_list = ['M5G','M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R','M9V','M9L','M5V','M1B', 'M5A']

test_dd = pd.DataFrame(columns=columnname)

for code in test_list:
  test_dd = test_dd.append(tor_location[tor_location['PostalCode']==code], ignore_index = True)
test_dd

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420
